In [ ]:
import requests
import pandas as pd

def get_openweathermap_data(api_key, state_name, country_code, start_date, end_date, variables, timezone="GMT"):
    """
    Fetch and process weather data from the OpenWeatherMap API using state name.

    Args:
        api_key (str): OpenWeatherMap API key.
        state_name (str): Name of the state.
        country_code (str): Country code (e.g., 'US' for the United States).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        variables (list): List of variables to fetch (e.g., 'temp', 'precipitation').
        timezone (str): Timezone of the data (default: "GMT").

    Returns:
        pd.DataFrame: DataFrame containing requested variables and their values.
    """
    # Step 1: Geocode the state to get latitude and longitude
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        "q": f"{state_name},{country_code}",
        "limit": 1,
        "appid": api_key
    }
    geocode_response = requests.get(geocode_url, params=geocode_params)
    if geocode_response.status_code != 200 or not geocode_response.json():
        raise Exception(f"Error fetching geocoding data: {geocode_response.text}")

    location_data = geocode_response.json()[0]
    latitude = location_data["lat"]
    longitude = location_data["lon"]

    # Step 2: Fetch weather data
    weather_url = "https://api.openweathermap.org/data/3.0/onecall"
    weather_params = {
        "lat": latitude,
        "lon": longitude,
        "exclude": "minutely,hourly,current,alerts",  # Exclude unwanted data
        "units": "metric",
        "appid": api_key
    }
    weather_response = requests.get(weather_url, params=weather_params)
    if weather_response.status_code != 200:
        raise Exception(f"Error fetching weather data: {weather_response.text}")

    weather_data = weather_response.json()

    # Step 3: Process daily data
    daily_data = weather_data.get("daily", [])
    data = {"date": []}
    for variable in variables:
        data[variable] = []

    for day in daily_data:
        date = pd.to_datetime(day["dt"], unit="s").date()
        if start_date <= str(date) <= end_date:
            data["date"].append(date)
            for variable in variables:
                if variable in day:
                    data[variable].append(day[variable])
                elif variable in day.get("temp", {}):  # Nested temperature data
                    data[variable].append(day["temp"][variable])
                elif variable in day.get("rain", {}):  # Nested precipitation data
                    data[variable].append(day["rain"])
                else:
                    data[variable].append(None)

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

# Example Usage
api_key = "YOUR_OPENWEATHERMAP_API_KEY"
state_name = "California"
country_code = "US"
start_date = "2024-11-01"
end_date = "2024-11-30"
variables = ["max", "min", "rain"]  # Example variables: temp max, temp min, precipitation

weather_data = get_openweathermap_data(api_key, state_name, country_code, start_date, end_date, variables)
print(weather_data)

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("open_weather_key")

state_name = "California"
country_code = "US"
start_date = "2024-11-01"
end_date = "2024-11-30"
variables = ["max", "min", "rain"]  # Example variables: temp max, temp min, precipitation

weather_data = get_openweathermap_data(api_key, state_name, country_code, start_date, end_date, variables)
print(weather_data)

Empty DataFrame
Columns: [date, max, min, rain]
Index: []
